In [ ]:
#!/usr/bin/env python3
import os, platform, subprocess, sys, requests

In [ ]:
"""
Finding and importing provided libraries regardless of system and platform type
"""
## find and import libraries relative to this file's location (regardless of script or notebook)
## platfrom specific naming already handled within the lib directory's __init__ file
if '__file__' not in globals():
    sys.path.append(os.path.dirname(os.path.abspath('').split('transactions')[0]))
else:
    sys.path.append(os.path.dirname(__file__).split('transactions')[0])

from lib.encoder  import encode_tx, encode_script
from lib.helper   import decode_address
from lib.hash     import hash160, hash256, sha256
from lib.rpc      import RpcSocket
from lib.rpcauth  import rpcauth
from lib.sign     import sign_tx

In [ ]:
'''
Step 0.0: setup our RPC socket like we did in the previous example
        If you have not done the previous example, now would be the time.
        Please copy in the variables you used (located conveniently at the end of the previous example)
'''

nodeIP = "127.0.0.1"
nodePort = "18444"
username = "user"
password = "password"
wallet = "test2"

rpc = RpcSocket({ 'wallet': wallet,
                  'username': username,
                  'password': password,
                  'url' : nodeIP,
                  'port': nodePort})
assert rpc.check()

In [ ]:
'''
Step 0.1: make sure you have cash. If you are broke, see previous example on how to get funds
'''
if float(rpc.call("getbalance")) == 0:
    raise Exception("get funds OR check if funds are still in a pending (ie. unconfirmed) state")
else:
    print("You have a balance of:", rpc.call("getbalance"), "BTC")

In [ ]:
## First, we will lookup an existing utxo,
## and use that to fund our transaction.
## Get a utxo for Alice.
alice_utxo = rpc.get_utxo(0)
print(alice_utxo)

In [ ]:
## Get a change address for Alice.
alice_change_txout = rpc.get_recv(fmt='base58')
alice_pubkey_hash  = decode_address(alice_change_txout['address'])

In [ ]:
## Get a payment address for Bob.
bob_payment_txout = rpc.get_recv(fmt='base58')
bob_pubkey_hash   = decode_address(bob_payment_txout['address'])

In [ ]:
## Calculate our output amounts.
fee = 1000
bob_recv_value = alice_utxo['value'] // 2
alice_change_value = alice_utxo['value'] // 2 - fee

In [ ]:
## The spending transaction.
atob_tx = {
    'version': 1,
    'vin': [{
        # We are unlocking the utxo from Alice.
        'txid': alice_utxo['txid'],
        'vout': alice_utxo['vout'],
        'script_sig': [],
        'sequence': 0xFFFFFFFF
    }],
    'vout': [
        {
            'value': bob_recv_value,
            'script_pubkey': ['OP_DUP', 'OP_HASH160', bob_pubkey_hash, 'OP_EQUALVERIFY', 'OP_CHECKSIG']
        },
        {
            'value': alice_change_value,
            'script_pubkey': ['OP_DUP', 'OP_HASH160', alice_pubkey_hash, 'OP_EQUALVERIFY', 'OP_CHECKSIG']
        }
    ],
    'locktime': 0
}


In [ ]:
## Serialize the transaction and calculate the TXID.
atob_hex  = encode_tx(atob_tx)
atob_txid = hash256(bytes.fromhex(atob_hex))[::-1].hex()

In [ ]:

## The redeem script is a basic Pay-to-Pubkey-Hash template.
redeem_script = f"76a914{alice_utxo['pubkey_hash']}88ac"

In [ ]:

## We are signing Alice's UTXO using BIP143 standard.
alice_signature = sign_tx(
    atob_tx,                # The transaction.
    0,                      # The input being signed.
    alice_utxo['value'],    # The value of the utxo being spent.
    redeem_script,          # The redeem script to unlock the utxo.
    alice_utxo['priv_key']  # The private key to the utxo pubkey hash.
)

In [ ]:

## Include the arguments needed to unlock the redeem script.
atob_tx['vin'][0]['witness'] = [ alice_signature, alice_utxo['pub_key'] ]

In [ ]:

print(f'''
## Pay-to-Pubkey-Hash Example ##
-- Transaction Id --
{atob_txid}
-- Alice UTXO --
     Txid : {alice_utxo['txid']}
     Vout : {alice_utxo['vout']}
    Value : {alice_utxo['value']}
     Hash : {alice_utxo['pubkey_hash']}
-- Sending to Bob --
  Address : {bob_payment_txout['address']}
    Coins : {bob_recv_value}
-- Change --
  Address : {alice_change_txout['address']}
      Fee : {fee}
    Coins : {alice_change_value}
-- Hex --
{encode_tx(atob_tx)}
''')

In [ ]:
tx_id = rpc.transact(encode_tx(atob_tx))


In [ ]:
## We can find this transaction and its status on the mem pool

api_url = "https://mempool.space/testnet/api/tx/" + tx_id
response = requests.get(api_url)
response.json()

In [ ]:
api_transact_url = "https://mempool.space/testnet/api/tx/" + tx_id + "/status"
response = requests.get(api_transact_url)
response.json()